In [1]:
'''
코버트 실행 환경
python==3.6
torch==1.7.1
transforers==3.5.1
'''
import sys
from os import path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from model.Embeder import SentenceEmbeder

In [2]:
project_dir = '/home/fhdufhdu/vscode/KoreanHistoryProject'
sys.path.append(project_dir)
from util.func import load_pickle, remove_duplicated_keyword, DependencyParser
# if __package__ is None:
#     current_path = path.dirname(path.dirname(path.abspath(
#         '/home/fhdufhdu/vscode/KoreanHistoryProject/keyword_extractor/keyword_extractor_ver_BERT.ipynb')))
#     sys.path.append(current_path)
#     from util.func import load_pickle, remove_duplicated_keyword, DependencyParser
# else:
#     from ..util.func import load_pickle, remove_duplicated_keyword, DependencyParser

# 데이터셋 생성(버트 데이터셋 x)
document_list = load_pickle('data/document_list.pickle')
title_list = load_pickle('data/title_list.pickle')

embeder = SentenceEmbeder(10)


[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-hnn (2.1.4) will be used.
[root] Java gateway started with port number 35969
[root] Callback server will use port number 25334
[py4j.java_gateway] An error occurred while trying to start the callback server (127.0.0.1:25334)
Traceback (most recent call last):
  File "/home/fhdufhdu/anaconda3/envs/sig/lib/python3.6/site-packages/py4j/java_gateway.py", line 2216, in start
    self.server_socket.bind((self.address, self.port))
OSError: [Errno 98] Address already in use


Py4JNetworkError: An error occurred while trying to start the callback server (127.0.0.1:25334)

In [3]:
# x_train, x_test, _, _ = train_test_split(document_list, document_list, test_size=0.2, shuffle=True, random_state=34)
doc_idx = 432
doc = document_list[doc_idx]
title = title_list[doc_idx]

In [4]:
parser = DependencyParser()
nouns = parser.get_cdks_with_parser(doc)
print(doc)
print(nouns)

계용묵은 일제강점기 『병풍에 그린 닭이』, 『백치아다다』 등을 저술한 소설가. 계용묵은 본관은 수안. 본명은 하태용. 평안북도 선천 출신. 아버지는 항교, 어머니는 죽산 박씨이며, 1남 3녀 중 장남이다. 대지주 집안에서 태어나 신학문을 반대하는 할아버지 밑에서 한문을 수학하였다. 향리의 삼봉공립보통학교에 다닐 때 순흥 안씨 안정옥과 혼인하였다.
['가', '군', '삼국시대', '고구', '장안성', '축성', '책', 'ㅁ', 'ㄴ', '관리', '키워드장안성축성평원왕계루부', '1964년', '평양', '중구역', '남문동', '발견', '내성', '성벽돌', '명문', '성곽', '축조시', '소형', '관등', '일정', '거리', '성벽', '축조', '감독하', '522년', '589년', '천도', '활동', '시기', '6세', '후반', '평원왕', '추정', '괘루개절', '가군', '출신지', '직명', '괘루', '5부', '계루부', '표', '관직명', '일종', '인민대학습당', '내부', '원위치']


In [5]:
# 문장, 후보 키워드 임베딩(KoBERT활용)
_, title_vector = embeder.encode([title])
_, noun_vectors = embeder.encode(nouns)

100%|██████████| 25/25 [00:00<00:00, 56.27it/s]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
title_vector = title_vector[0]
doc_embedding_ = [[value.item() for value in title_vector]]
candicates_embedding_ = [[value.item() for value in noun_vector]for noun_vector in noun_vectors]
distances = cosine_similarity(doc_embedding_, candicates_embedding_)
print(distances.argsort())
keywords = [nouns[index] for index in distances.argsort()[0]]  

[[41 21 27  6 40 22 29  4 37 14 43  0 32 18 35 48 30 47  9 10 36 23 19  2
  13 34 24 38  7  8 39 49 11 15 17 31 44 33 20 28  3 26 25 45 46  1 16 42
  12  5]]


In [7]:
print(title, ":", doc)
keywords.reverse()
print(keywords)

계용묵 : 계용묵은 일제강점기 『병풍에 그린 닭이』, 『백치아다다』 등을 저술한 소설가. 계용묵은 본관은 수안. 본명은 하태용. 평안북도 선천 출신. 아버지는 항교, 어머니는 죽산 박씨이며, 1남 3녀 중 장남이다. 대지주 집안에서 태어나 신학문을 반대하는 할아버지 밑에서 한문을 수학하였다. 향리의 삼봉공립보통학교에 다닐 때 순흥 안씨 안정옥과 혼인하였다.
['축성', '평양', '5부', '내성', '군', '일종', '관직명', '성벽', '축조', '고구', '522년', '축조시', '6세', '표', '활동', '성벽돌', '발견', '1964년', '원위치', '출신지', 'ㄴ', 'ㅁ', '가군', '거리', '후반', '중구역', '삼국시대', '성곽', '일정', '추정', '키워드장안성축성평원왕계루부', '관리', '인민대학습당', '천도', '내부', '평원왕', '명문', '시기', '가', '계루부', '남문동', '괘루개절', '장안성', '589년', '관등', '직명', '책', '감독하', '소형', '괘루']


In [8]:
# from gensim.models import Word2Vec # candidates = [get_cdks_with_parser(doc) for doc in [ document_list[i] for i in range(10)]] # embedding_model = Word2Vec(candidates, size=100, #                            window=5, min_count=1, workers=4, iter=10, sg=1) # words = get_cdks_with_parser(document_list[0]) # similar_list = [] # for word in words: #     similar_list.append( #         (word, embedding_model.wv.similarity(w1=title[0], w2=word))) # similar_list = sorted(similar_list, key=lambda sim: sim[1], reverse=True) # print(similar_list)

In [9]:
# print(title[0])
# print([word for word, similar in [similar_list[i] for i in range(5)]])
